## 今天的作業
在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。

* Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。  
* Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。  
*  Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?

In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
%matplotlib inline


#顯示所有行
pd.set_option('display.max_columns', None)
#顯示所有列
pd.set_option('display.max_rows', None)
#顯示行設定
pd.set_option('max_colwidth',100)

import pingouin as pg
import researchpy   
 

0.23.1


## 讀入資料

In [2]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId      891 non-null int64
Survived         891 non-null int64
Pclass           891 non-null int64
Name             891 non-null object
Sex              891 non-null object
Age              714 non-null float64
SibSp            891 non-null int64
Parch            891 non-null int64
Ticket           891 non-null object
Fare             891 non-null float64
Cabin            204 non-null object
Embarked         889 non-null object
Survived_cate    891 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [6]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除
complete_data=df_train[['Age','Survived_cate','Sex']].dropna()
display(complete_data.head(5))

,Age,Survived_cate,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male


### Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。


In [7]:
## 數值到分類的映射
# 運用 apply function  做變數轉換
def Age_above65_(x): 
    if(x>=65):
        return('Y')
    else: 
        return('N')
    
complete_data['age_category']=complete_data['Age'].apply(Age_above65_)

display(complete_data.head(5))

,Age,Survived_cate,Sex,age_category
0,22.0,0,male,N
1,38.0,1,female,N
2,26.0,1,female,N
3,35.0,1,female,N
4,35.0,0,male,N


### Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。
* 暗示: 觀看下面影片找答案，https://www.youtube.com/watch?v=X2d-wUt5azY

In [20]:
def female65(df):
    if df['age_category'] == 'Y' or df['Sex'] == 'female':
        return 'Y'
    else:
        return 'N'
    
complete_data['Age_above65_female'] = complete_data.apply(female65, axis=1)
display(complete_data.head(10))

,Age,Survived_cate,Sex,age_category,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N
6,54.0,0,male,N,N
7,2.0,0,male,N,N
8,27.0,1,female,N,Y
9,14.0,1,female,N,Y
10,4.0,1,female,N,Y


### Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?
* 提示:    
  首先觀察一下這些變數的資料型態後，再來想要以哪一種判斷倆倆的相關性。     

In [11]:
contTable1 = pd.crosstab(complete_data['Survived_cate'], complete_data['age_category'])
contTable1

age_category,N,Y
Survived_cate,,
0,414,10
1,289,1


In [12]:
df1 = min(contTable1.shape[0], contTable1.shape[1]) - 1
df1

1

In [13]:
crosstab1, res1 = researchpy.crosstab(complete_data['Survived_cate'], complete_data['age_category'], test='chi-square')
print(res1)

                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    4.6035
1                    p-value =    0.0319
2               Cramer's phi =    0.0803


In [15]:
def judgment_CramerV(df,V):
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

judgment_CramerV(df1,res1.loc[2,'results'])

'negligible'

In [21]:
contTable2 = pd.crosstab(complete_data['Survived_cate'], complete_data['Age_above65_female'])
print(contTable2)
df2 = min(contTable2.shape[0], contTable2.shape[1]) - 1
print(df2)
crosstab2, res2 = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Age_above65_female'], test='chi-square')
print(res2)

Age_above65_female    N    Y
Survived_cate               
0                   350   74
1                    92  198
1
                Chi-square test   results
0  Pearson Chi-square ( 1.0) =   188.6219
1                    p-value =     0.0000
2               Cramer's phi =     0.5140


In [22]:
judgment_CramerV(df2,res2.loc[2,'results'])

'large'

## Ans: Age_above65_female 和 Survived_cate 相關性較高